# 澳門日報爬蟲

 


In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
from lxml import etree
import pandas as pd 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [ ]:
# 一年中所有的日期
def get_date_list(start,end):
    start_year, start_month, start_day=start.split("/")
    end_year, end_month, end_day= end.split("/")
    begin = datetime.date(int(start_year), int(start_month), int(start_day))
    end = datetime.date(int(end_year), int(end_month), int(end_day))
    delta = datetime.timedelta(days=1)
    days = []
    while begin <= end:
        days.append(begin.strftime("%Y-%m-%d"))
        begin += delta
    date_list=list(map(lambda x: x.replace("-","/" ).replace("/","-",1), days))
    return date_list

In [ ]:
def get_content(date,node_name,title,link):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
    resp = requests.get(url=link, headers=headers)
    resp.encoding = 'utf-8'
    html = etree.HTML(resp.text)
    try:
        content='\n'.join(html.xpath('//div[@id="ozoom"]//p//text()'))
    except AttributeError:
        content=''       
    return [date,node_name,title,link,content]

def news(start, end):
    date_list=get_date_list(start, end)
    #print(date_list[0:5])
    result=[]
    for date in date_list:
        url=f'http://www.macaodaily.com/html/{date}/node_1.htm'
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
        resp=requests.get(url=url,headers=headers)
        resp.encoding='utf-8'
        html=etree.HTML(resp.text)
        #每个版块
        h_list=html.xpath('//div[@class="list"]/div/h4')
        d_list=html.xpath('//div[@class="list"]/div/div')
        for h ,d in zip(h_list,d_list):
            node_name=''.join(h.xpath('.//a/text()'))
            #print(node_name)
            li_list=d.xpath('./ul/li')
            for li in li_list:
                title=''.join(li.xpath('.//a/text()'))
                link=f'http://www.macaodaily.com/html/{date}/'+''.join(li.xpath('.//a/@href'))
                print(date,node_name,title,link)
                r=get_content(date, node_name, title, link)
                result.append(r)
    news_df=pd.DataFrame(result)
    news_df.columns=['date', 'node_name', 'title', 'link','content']
    print(news_df)
    return news_df
       

In [ ]:
start ="2020/10/01"
end = "2020/10/02"
news_df=news(start, end)

In [ ]:
#news_df.to_excel(f"{start}至{end}".replace('/','')+'.xlsx',index=False)
#news_df.to_excel("test.xlsx",index=False)
news_df.to_pickle('./news_df-oneday.pkl')
#news_df = pd.read_csv('./news_df.pkl')

In [ ]:
import nltk
nltk.download('stopwords')
#stopwords_list = stopwords.words('english')
from nltk.corpus import names, stopwords, words
stopwords_list = stopwords.words('chinese')

In [ ]:
import string
# Take a string as input and perform the corresponding cleaning
def clean(doc): 
    # Clean up html tags
    doc = doc.replace("</br>", " ")
    # remove punctuation and numbers
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    # remove stopwords
    doc = " ".join([token for token in doc.split() if token not in stopwords_list]) 
    return doc

In [ ]:
news_df['news_type'] = news_df['node_name'].apply(lambda x: x.split('：')[1])
news_df['content'] = news_df['content'].apply(lambda x: x.replace(u'\xa0', u' '))
news_df['content'] = news_df['content'].apply(clean)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
a=news_df['news_type'].unique()
news_df['news_type_num'] = labelencoder.fit_transform(news_df['news_type'])
b=news_df['news_type_num'].unique()
label=news_df['news_type_num'].value_counts().shape[0]

nodenum=[]
for i,j in zip(a,b):
  nodenum.append([i,j])
nodenum

news_df['news_type_num'][0]
news_df['content'][0]
int_label=news_df['news_type_num'].value_counts().shape[0]
int_label

In [ ]:
condition1 = news_df['news_type']=='澳聞'
macau_news = news_df[condition1]
macau_news_content = macau_news['content']
a=macau_news_content.apply(lambda x: x.split('【')[0])
b=macau_news_content.apply(lambda x: x.split('【')[1:-1])

In [ ]:
macau_news_content